# Llama_Indexを試す

In [2]:
import llama_index
llama_index.__version__

'0.8.24.post1'

## PDFを読み込む
- 読み込み可能なアプリケーション
  - dvipdfmx
  - word
- 読み込みがうまくかないアプリケーション
  - Adobe
  - iText

- 使用データ
  - [STクラウドサーバーサービス契約約款](https://www.stnet.co.jp/cmmn/data/stcloud.pdf)
  - [インターネットセキュリティサービス契約約款
(https://www.stnet.co.jp/cmmn/data/inet.pdf)
  - [ビジネス光ネットサービス契約約款](https://www.stnet.co.jp/cmmn/data/oshigotonet.pdf)]

In [2]:
# from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index import download_loader

CJKPDFReader = download_loader("CJKPDFReader")
loader = CJKPDFReader()
documents = loader.load_data(file="pdf_data/data_0.pdf")
print(len(documents))

1


In [3]:
with open("./pdf_to_text/pdf_to_text_0.txt",encoding="utf-8",mode="w") as f:
    f.write(documents[0].text) 

In [87]:
documents[0].to_dict().keys()

dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'hash', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])

In [51]:
text_data = documents[0].text.split("\n\n")
text_data = [text.replace("\n","") for text in text_data] 

In [52]:
len(text_data)

1088

In [88]:
documents[0].text[:200]

'IA0248_02tanaka (2021-03-05 15:41)\n\n一般論文\n\n雑談対話応答における連続する事態の一貫性と対話継続性の関係\n\n田中\u3000翔平 †・吉野幸一郎 †,††・須藤\u3000克仁 †・中村\u3000\u3000哲 †\n\n雑談対話システムの評価指標として，ユーザとの対話を継続させる働きを表す，対話\n継続性が挙げられる．対話モデルの先行研究において，対話継続性の向上には，シ\nステム発話の一貫性が重要であ'

## Qdrantの設定

In [3]:
import qdrant_client
import logging
from llama_index.storage.storage_context import StorageContext
import sys

client = qdrant_client.QdrantClient(host='qdrant', port=6333)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

vector_store = QdrantVectorStore(client=client, collection_name="")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

NameError: name 'QdrantVectorStore' is not defined

## ベクトル化
### note
- 文分割は読点等のsplit程度
- データごとに処理が必要

In [35]:
paragraph_separator = "\n\n" # 段落分割
chunk_size = 1024 #チャンク（トークン）数
chunk_overlap = 20 # 前のチャンクをどのくらい含めるか
secondary_chunking_regex = '[^,.．;。]+[,.．;。]?'# 文分割

In [36]:
from llama_index.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex,ServiceContext
import tiktoken

# ノードパーサーの準備
text_splitter = SentenceSplitter(
    paragraph_separator=paragraph_separator,
    secondary_chunking_regex = secondary_chunking_regex,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
)
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter
)
service_context = ServiceContext.from_defaults(
    node_parser=node_parser
)
service_context.node_parser

SimpleNodeParser(text_splitter=SentenceSplitter(chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n', secondary_chunking_regex='[^,.．;。]+[,.．;。]?', chunking_tokenizer_fn=<function sent_tokenize at 0x7f4b8e8ef6a0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f4bcf9d0d50>, tokenizer=functools.partial(<bound method Encoding.encode of <Encoding 'gpt2'>>, allowed_special='all')), include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f4bcfb6ab90>)

In [37]:
node_list = service_context.node_parser.get_nodes_from_documents(documents)
enc = tiktoken.get_encoding("gpt2")
print(len(node_list))
for node in node_list:
    print(len(enc.encode(node.text)))
    print(len(node.text))
    print(node.text.replace("\n",""))
    break

52
946
600
IA0248_02tanaka (2021-03-05 15:41)一般論文雑談対話応答における連続する事態の一貫性と対話継続性の関係田中　翔平 †・吉野幸一郎 †,††・須藤　克仁 †・中村　　哲 †雑談対話システムの評価指標として，ユーザとの対話を継続させる働きを表す，対話継続性が挙げられる．対話モデルの先行研究において，対話継続性の向上には，システム発話の一貫性が重要であると考えられている．そこで本論文では，対話モデルより生成された応答候補を，対話中に含まれる事態の一貫性に基づいてリランキングする手法を提案する．提案手法は対話に含まれる事態の一貫性（「ストレスが溜まる」と「発散する」は関連した事態である，など）を考慮することで，選択される応答の一貫性，対話継続性の向上を図る．本研究では異なる 2 つの手法を考案した．一つ目の手法は統計的に獲得された因果関係ペアとのマッチングにより，対話中の事態の一貫性を考慮し，二つ目の手法は Coherence Model によって，対話の一貫性を考慮する．自動評価の結果，これらの手法では応答中の単語選択の観点では一貫性は向上していることが確認された．一方で，人手評価の結果では，応答の主観的な一貫性は明確に向上しないものの，一つ目の方法により対話継続性が向上するという，一見して矛盾する結果が確認された．


In [ ]:
index = VectorStoreIndex.from_documents(documents=documents,vector_store=vector_store,service_context=service_context)